In [ ]:
# Library w/ machine learning models
!pip install mllibs


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
import missingno as msno

from catboost import CatBoostRegressor as CAT
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error as mse
#from mllibs.kriging_regressor import Kriging
from pykrige.ok import OrdinaryKriging

lst_color = ['#B1D784','#2E8486','#004379','#032B52','#EAEA8A']

In [43]:
class TS:

    # class instantiation : read class & store 
    def __init__(self,path):
    
        ''' Read Dataset '''
        self.df = pd.read_csv(path)
        del self.df['Unnamed: 0']
        
        # get additional response function
        self.df['L/D'] = self.df['cl']/self.df['cd'] # Get lift/drag 
                 
    ''' [SUBSET] Show a specific design row '''
    def get_id(self,design_id):
        return self.df[self.df['design'] == design_id]
    
    ''' [SUBSET] Get all designs for a specific angle '''
    def get_aoasubset(self,angle_id=None):
        if(angle_id is None):
            return self.df.aoa.unique()
        else:
            return dict(tuple(self.df.groupby('aoa')))[angle_id]
    
    ''' [PLOT] 1. Plot a design variable X against the response variables cl,cd '''
    


    def plot_X_clcd(self, feature, model=False, xmin=-15, xmax=25):
        fig = make_subplots(rows=1, cols=2)

        # create interpolation model for design variable vs response variables
        if model:

            # lift
            ok_lift = OrdinaryKriging(
                self.df[feature].values,
                np.zeros_like(self.df[feature].values),  # Using zeros as y-coordinates for 1D interpolation
                self.df['cl'].values,
                variogram_model='linear'
            )
            Xm = np.arange(xmin, xmax, 1)
            ym, ss = ok_lift.execute('grid', Xm, np.array([0.0]))  # Grid interpolation

            fig.add_trace(go.Scatter(x=Xm, y=ym.flatten(), mode='lines',
                                    marker=dict(color='#454545'),
                                    name='<b>LIFT MODEL</b>'),
                        row=1, col=1)

            # drag
            ok_drag = OrdinaryKriging(
                self.df[feature].values,
                np.zeros_like(self.df[feature].values),  # Using zeros as y-coordinates for 1D interpolation
                self.df['cd'].values,
                variogram_model='linear'
            )
            ym, ss = ok_drag.execute('grid', Xm, np.array([0.0]))  # Grid interpolation

            fig.add_trace(go.Scatter(x=Xm, y=ym.flatten(), mode='lines',
                                    marker=dict(color='#454545'),
                                    name='<b>DRAG MODEL</b>'),
                        row=1, col=2)

        # Plot Designs
        fig.add_trace(go.Scatter(x=self.df[feature], y=self.df['cl'],
                                mode='markers', marker=dict(color='#127CF3'),
                                text=self.df['design'], name='<b>LIFT</b>'),
                    col=1, row=1)

        fig.add_trace(go.Scatter(x=self.df[feature], y=self.df['cd'],
                                mode='markers', marker=dict(color='#DE4747'),
                                text=self.df['design'], name='<b>DRAG</b>'),
                    col=2, row=1)

        # Plot Aesthetics
        fig.update_layout(template='plotly_white',
                        font=dict(family='sans-serif', size=12),
                        title=f'<b>{feature.upper()}</b> | LIFT & DRAG DEPENDENCY ON {feature.upper()}')
        fig.update_layout(margin=dict(l=40, r=40, b=40), height=400)
        fig.update_traces(marker=dict(size=5, line=dict(width=1.0, color='black')))
        fig.update_xaxes(title=f'<b>{feature.upper()}</b>')
        fig.show()

    ''' [PLOT] 2. Plot Parallel Coordinates using Plotly Figure Factory '''
    
    def plot_par_coord(self,ldf,colour='L/D'):
        fig = go.Figure(data=
            go.Parcoords(
                line = dict(color = ldf[colour].round(4),
                           colorscale = px.colors.sequential.Jet),
                dimensions = [dict(label=col, values=ldf[col]) 
                              for col in ldf.columns.tolist()]
            )
        )
        fig.update_layout(template='plotly_white',
                          title='<b>PARALLEL COORDINATES</b> | AIRFOIL DESIGN EXPLORATION',
                          font=dict(family='sans-serif',size=12),
                          height=450,margin=dict(l=40, r=40, t=120, b=40))
        # fig.write_html("parallel_coord.html")
        fig.show()
        
    ''' [PLOT] 3. Plot Drag Polar Curve '''
    
    def plot_polar(self):

        fig = px.scatter(self.df, x="cd", y="cl",hover_name='design',
                         color=self.df['L/D'],opacity=0.9)
        fig.update_layout(margin=dict(t=100),height=500)
        fig.update_layout(template='plotly_white',
                          title='<b>DRAG POLAR</b> | ALL DESIGNS',
                          font=dict(family='sans-serif',size=12))
        fig.update_traces(marker=dict(size=5,line=dict(width=1.0,color='black')))
        fig.update_xaxes(range=[0,0.5]); fig.update_yaxes(range=[-1.2,2.5])
        fig.show()
        
    ''' [PLOT] 4. Plot Drag Polar Curve w/ Angle Subsets '''
    
    def plot_angle_polar(self):
        
        angles = self.get_aoasubset(angle_id=None) # get all available angles
        
        fig = go.Figure()
        for angle in angles:
            aoa_subset = self.get_aoasubset(angle_id=angle)
            fig.add_trace(go.Scatter(x=aoa_subset['cd'],y=aoa_subset['cl'],
                                     mode='markers',text=aoa_subset['design'],
                                     name=f'<b>ANGLE : {angle}</b>'))

        fig.update_layout(template='plotly_white',title='<b>DRAG POLAR</b> |' \
                          ' AOA VARIATION',height=500,
                          font=dict(family='sans-serif',size=12),
#                           margin=dict(l=40, r=40, t=120, b=40)
                         )
        fig.update_xaxes(range=[0,0.5]); fig.update_yaxes(range=[-1.2,2.5])
        fig.update_traces(marker=dict(size=5,line=dict(width=1.0,color='black')))
        fig.show()
    
    ''' [PLOT] 5. Plot Scatter Matrix '''
    
    def plot_scat_mat(self,ldf=None,dim=None,colour=None,hov_name=None,title=None):

        fig = px.scatter_matrix(ldf,dimensions=dim,opacity=0.5,
                                color=colour,hover_name=hov_name,height=1200)
        fig.update_traces(marker=dict(size=5,line=dict(width=0.5,color='black')))
        fig.update_layout(template='plotly_white',
                          title='<b>SCATTER MATRIX</b> | AIRFOIL DESIGN EXPLORATION',
                          font=dict(family='sans-serif',size=12),
                          margin=dict(l=20, r=20, t=120, b=20))
        fig.update_traces(diagonal_visible=False)
        fig.update_layout(showlegend=False)
        fig.show()

In [44]:
n2412_ts = 'n2412_optimisation.csv'
data = TS(n2412_ts)

data.df.head()

,id,design,cd,cl,performance,aoa,maxcamber,maxcamberposition,thickness,L/D
0,1,Design 1,0.017558,0.815590,-1.012540e-08,5,2.00,40,12.0,46.451719
1,2,Design 2,0.250152,1.802180,-1.203760e+01,25,7.77,49,25.8,7.204340
2,3,Design 3,0.135045,-0.315530,-1.003670e+03,-11,6.05,16,8.1,-2.336480
3,4,Design 4,0.039020,-0.635606,-4.042950e+03,-15,5.18,33,32.9,-16.289236
4,5,Design 5,0.035562,1.047800,-7.406950e-01,0,9.50,65,22.3,29.464117


In [45]:
print(f'trade-study cases: {data.df.shape[0]}')

trade-study cases: 480


In [46]:
data.df.describe()

,id,cd,cl,performance,aoa,maxcamber,maxcamberposition,thickness,L/D
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,240.500000,0.072042,1.162044,-121.129251,6.812500,5.761188,49.335417,11.925625,26.064067
std,138.708327,0.084987,0.742741,1045.575227,8.046203,3.029010,13.390630,8.854631,17.163784
min,1.000000,0.005715,-1.163990,-13553.200000,-15.000000,0.000000,0.000000,1.000000,-33.429812
25%,120.750000,0.018055,0.602206,-4.045338,1.000000,2.675000,44.000000,2.575000,12.940457
50%,240.500000,0.044583,1.231180,-0.832947,5.000000,6.910000,49.000000,12.100000,27.673820
75%,360.250000,0.092478,1.820742,-0.199992,13.000000,8.512500,56.000000,17.200000,38.965487
max,480.000000,0.601579,2.360070,0.235968,25.000000,9.500000,90.000000,40.000000,60.479830


In [47]:
''' Plot Univariate Statistics using Plotly '''
# Input Feature Matrix Dataframe

def px_stats(df, n_cols=4, to_plot='box',height=800):
    
    ldf = df.select_dtypes(include=['float64','int64'])    
#     ldf = df.select_dtypes(exclude=['float64','int64'])

    numeric_cols = ldf.columns
    n_rows = -(-len(numeric_cols) // n_cols)  # math.ceil in a fast way, without import
    row_pos, col_pos = 1, 0
    fig = make_subplots(rows=n_rows, cols=n_cols,subplot_titles=numeric_cols.to_list())
    
    for col in numeric_cols:
        if(to_plot is 'histogram'):
            trace = go.Histogram(x=ldf[col],showlegend=False,nbinsx=40)
        else:
            trace = getattr(px, to_plot)(ldf[col],x=ldf[col])["data"][0]
            
        if col_pos == n_cols: 
            row_pos += 1
        col_pos = col_pos + 1 if (col_pos < n_cols) else 1
        fig.add_trace(trace, row=row_pos, col=col_pos)

    fig.update_layout(template='plotly_white',
                      title=f'UNIVARIATE FEATURE DISTRIBUTION',
                      font=dict(family='sans-serif',size=12))
    
    if(to_plot is 'histogram'):
        fig.update_traces(marker=dict(line=dict(width=1, color='white')))
    fig.update_layout(height=height);fig.show()

<>:15: SyntaxWarning:

"is" with a literal. Did you mean "=="?

<>:29: SyntaxWarning:

"is" with a literal. Did you mean "=="?

<>:15: SyntaxWarning:

"is" with a literal. Did you mean "=="?

<>:29: SyntaxWarning:

"is" with a literal. Did you mean "=="?

C:\Users\junej\AppData\Local\Temp\ipykernel_24496\1801499270.py:15: SyntaxWarning:

"is" with a literal. Did you mean "=="?

C:\Users\junej\AppData\Local\Temp\ipykernel_24496\1801499270.py:29: SyntaxWarning:

"is" with a literal. Did you mean "=="?



In [48]:
px_stats(data.df.drop(['id'],axis=1),to_plot='histogram',height=500)

In [49]:
#pip install nbformat --upgrade

In [50]:
data.plot_X_clcd('maxcamber',model=True,xmin=0,xmax=11)
data.plot_X_clcd('maxcamberposition',model=True,xmin=0,xmax=90)
data.plot_X_clcd('thickness',model=True,xmin=0,xmax=40)

In [51]:
data.plot_X_clcd('aoa',model=True,xmin=-15,xmax=26)

In [52]:
data.plot_polar()

In [53]:
df1 = data.df.sort_values(by='L/D',ascending=False).copy()
del df1['performance']; del df1['design']
df3 = df1.astype('float').round(4)
    
data.plot_par_coord(df3,'L/D')

In [54]:
# Choose a high L/D ratio subset 
df1 = data.df.copy()
df1['HIGH_LD'] = df1['L/D'] > 55.0 
del df1['performance']; del df1['id']
    
# fit in as many relevant features as possible
lst = df1.columns.values.tolist()
lst.remove('HIGH_LD'); lst.remove('design')
    
# plot scatter matrix
data.plot_scat_mat(ldf=df1,
                   dim=lst,colour='HIGH_LD',
                   hov_name=df1.design,
                   title='Design Matrix')

In [55]:
models = []
models.append(('CAT',CAT(n_estimators=1000,silent=True)))
models.append(('XGB',XGBRegressor(n_estimators=1000,learning_rate=1e-4)))
models.append(('RF',RF(n_estimators=1000)))
models.append(('BAG',BaggingRegressor(n_estimators=1000)))
models.append(('ADA',AdaBoostRegressor(n_estimators=1000,learning_rate=1e-4)))
models.append(('GBR',GBR(n_estimators=1000,learning_rate=1e-4)))
models.append(('ETR',ExtraTreesRegressor(n_estimators=1000)))
models.append(('KRN',KNR(n_neighbors=10)))
models.append(('SVR',SVR()))

In [56]:
''' Evaluation Class '''
class evaluation():

    # Class instantiation
    def __init__(self,ldf=None,cv=5,shuffle=False,models=None):
        
        self.ldf = ldf
        
        self.shuffle = shuffle
        self.cv = cv
        self.models = models
        
        self.dic_truth = {}  # store kfold truth values
        self.dic_eval = {}  # store kfold model values        
        self.dic_trerr = {}
        self.dic_teerr = {}
        
    # Show the DataFrame used in Kfold evaluation
    def show_ldf(self):
        display(self.ldf.head())
        
    ''' [MAIN] KFOLD Model Evaluation '''
    def kfold(self,target='target'):

        # cycle through all defined models
        for model in self.models:
        
            # Split feature/target variable
            y = self.ldf[target].copy()
            X = self.ldf.copy()
            del X[target]     # remove target variable

            ''' Fit Model on Train Set & Make Evaluation Set Prediction '''
            # RMSE Evaluation metric is used 

            kf = KFold(n_splits=self.cv,shuffle=self.shuffle)    
            X = X.values; y = y.values

            kf.get_n_splits(X)
            lerr_train = []; lerr_eval = []; temp_ym = []; temp_y = []
            for train_index, eval_index in kf.split(X):
                
                # split kfold
                X_train, X_eval = X[train_index], X[eval_index]
                y_train, y_eval = y[train_index], y[eval_index]
                
                # fit & predict
                model[1].fit(X_train,y_train)        
                ym_train = model[1].predict(X_train).tolist()  
                ym_eval = model[1].predict(X_eval).tolist()
                
                # evaluation metric rmse
                err0 = mse(y_train, ym_train,squared=False)
                err1 = mse(y_eval,ym_eval,squared=False)
                
                # store error/values
                lerr_eval.append(err1); lerr_train.append(err0)
                temp_ym.append(ym_eval); temp_y.append(y_eval)
                
            # Store Results
            self.dic_teerr[model[0]] = lerr_eval
            self.dic_trerr[model[0]] = lerr_train
            self.dic_truth[model[0]] = temp_y
            self.dic_eval[model[0]] = temp_ym
            
    ''' PLOTS '''
    # Plot kfold RMSE errors
    
    def plot_err(self):
        
        fig = px.box(evals.dic_trerr)
#         fig = px.strip(evals.dic_trerr)
        fig.update_layout(template='plotly_white',height=300,
                          title=f'<b>TRAINING RMSE</b>',
                          font=dict(family='sans-serif',size=11))
        fig.update_traces(width=0.2,marker_color='#2DB1AB')
        fig.show()
        
        fig = px.box(evals.dic_teerr)
#         fig = px.strip(evals.dic_teerr)
        fig.update_layout(template='plotly_white',height=300,
                          title=f'<b>TEST RMSE</b>',
                          font=dict(family='sans-serif',size=11))
        fig.update_traces(width=0.2,marker_color='#273746')
        fig.show()
        
    # Plot Target Variables (truth & model values)
    # model_id : id number as ordered in define_models, kfold : id number kfold target 
    def plot_evals(self,model_id,kfold):
        fig = go.Figure()
        fig.add_trace(go.Bar(x=[i for i in range(0,len(self.dic_eval[model_id][kfold]))],
                             y=self.dic_eval[model_id][kfold],marker_color='#2DB1AB',
                             opacity=1.0,showlegend=False,name='evaluation'))
        fig.add_trace(go.Bar(x=[i for i in range(0,len(self.dic_truth[model_id][kfold]))],
                             y=self.dic_truth[model_id][kfold],marker_color='#273746',
                             opacity=1.0,showlegend=False,name='truth'))
        fig.update_layout(template='plotly_white',
                          title=f'<b>EVALUATION SET [{kfold+1}/{self.cv}]</b> | '
                          + f'{model_id} Predictions',
                          font=dict(family='sans-serif',size=11),
#                           margin=dict(l=20, r=20, t=80, b=20),
                          xaxis=dict(rangeslider=dict(visible=True)),
                          height=300)
#         fig.update_layout(boxgroupgap=0.2, boxgap=0.8)
        fig['layout']['xaxis'].update(title='', range=[0,50], autorange=False)
        fig.show()
        

In [57]:
''' Prepare DataFrame '''
model_df = data.df.copy()
del model_df['id'] 
model_df.index = model_df['design']  # set index
del model_df['design']; del model_df['cl']; del model_df['cd'] # remove leaky/irrelovant

In [58]:
''' KFOLD Evaluation '''
evals = evaluation(model_df,models=models)  # instantiate evaluation class
evals.kfold(target='L/D')     # evaluate kfold
evals.plot_err()   # plot RMSE error for all folds